In [20]:
!pip install pymysql

In [1]:
'''
使用說明:
#  僅供以 pandas.df 對專題用的 MySQL db 進行資料的輸出,輸入 (不得修改SQL內容)
1. 匯入class --> from connect_MySQL import MysqlDataFrame
2. 需要匯入packages --> pymysql, pandas, sqlalchemy.create_engine, logging
3. 建立 MysqlDataFrame 物件 --> df = MysqlDataFrame('user', 'pwd', 'db')
4  print(df) --> 可印出使用說明
5. df.show_info() --> 顯示現有的 databases, tables 等資訊
6. df.get_pandas_df("table名稱") --> 得到 pandas.df
7. df.use_sql_query("自己輸入的SQL語法") --> 得到 pandas.df
8. insert_pandas_df(df, "table名稱") --> 將 pandas.df 填入 MySQL
9. convert_str_to_list(df, column名稱) --> 將 series 內的 str 轉成 list
'''

import pandas as pd
import logging
from sqlalchemy import create_engine


class MysqlDataFrame:
    def __init__(self, user, pwd, db='tfb1031_project', ip='10.2.16.174'):
        self.user = user
        self.pwd = pwd
        self.db = db
        self.__conn_ip = ip
        self.__stopWords = [
            'alter', 'update', 'delete', 'drop', 'insert',
            'table', 'database'
            ]
        self.__sysDatabase = [
            'information_schema', 'performance_schema', 'mysql',
            'sakila', 'sys', 'world'
            ]

    def __repr__(self):
        return '''
        1. Call "show_info()" to get the db & table list
        2. Call "get_pandas_df()" to get a pd.DataFrame from MySQL table
        3. Call "use_sql_query()" to get a pd.DataFrame with customized SQL 
        4. Call "insert_pandas_df()" to insert pd.DataFrame to MySQL 
        5. Call "convert_str_to_list()" to convert string to list, return pd.series
        '''

    def __create_conn(self):
        try:
            engine = create_engine(
                f'mysql+pymysql://{self.user}:{self.pwd}@{self.__conn_ip}:3306/{self.db}'
                )
            return engine
        except Exception as err:
            print(logging.error(str(err)))

    def show_info(self):
        engine = self.__create_conn()

        # Get db list
        sql_db = 'SHOW databases;'
        query_db = engine.execute(sql_db).fetchall()
        db = [i[0] for j, i in enumerate(query_db) if i[0] not in self.__sysDatabase]
        # Get table list
        sql_table = 'SHOW tables;'
        query_table = engine.execute(sql_table).fetchall()
        table = [i[0] for i in query_table]

        information = f'Use db = {self.db}\nUser = {self.user}\nDatabase list = {db}\nTable list =  {table}'
        return print(information)

    def get_pandas_df(self, table='test'):
        engine = self.__create_conn()
        sql = f'select * from {table};'
        try:
            df = pd.read_sql_query(sql, engine)
            return df
        except Exception as err:
            print(logging.error(str(err)))

    # Use user-defined SQL
    def use_sql_query(self, input_sql):
        engine = self.__create_conn()
        for word in self.__stopWords:
            if word in input_sql:
                return print("Please don't alter the data")
        try:
            df = pd.read_sql_query(input_sql, engine)
            return df
        except Exception as err:
            print(logging.error(str(err)))

    def insert_pandas_df(self, df, table):
        engine = self.__create_conn()
        try:
            df.to_sql(table, engine, if_exists='append', index=0)
        except Exception as err:
            print(logging.error(str(err)))

    def convert_str_to_list(self, df, column):
        import ast
        return df[f'{column}'].apply(lambda x: ast.literal_eval(x))

In [2]:
df = MysqlDataFrame('account', 'password')
df.show_info()

Use db = tfb1031_project
User = tfb1031_19
Database list = ['app_accounting', 'test', 'tfb1031_project']
Table list =  ['author_feature', 'bnb', 'bnb_article', 'bnb_recommend_w2v', 'res_recommend_w2v', 'restaurant', 'test_restaurant', 'user_cost', 'user_info', 'user_questionnaire']


In [7]:
a = df.get_pandas_df('bnb_article')

In [15]:
# a.head()
# a.shape

,bnb_art_id,title,content,date,art_url,image_url,aut_id,bnb_id
0,40000001,台北西門町住宿|捷絲旅|四星級飯店|只要平價即可享受|逛完西門町|直接走回,"['台北美食', '美式校園輕食餐廳', '農人餐桌', '親子餐廳', '徹思叔叔咖啡廳'...",2021-10-12,https://zh472.pixnet.net/blog/post/333805750-%...,['https://pic.pimg.tw/zh472/1631501995-1172602...,30005027.0,NaN
1,40000002,天成集團住宿趣|天成文旅|華山町|忠孝新生站|華山文創園區|結合藝廊與旅,"['天成文旅', '華山町', '位於華山文化創意園區附近', '是一棟充滿故事的建築物',...",2021-10-01,https://laurasweet0712.pixnet.net/blog/post/22...,['https://paper-attachments.dropbox.com/s_523C...,30000558.0,NaN
2,40000003,五倍券|台北住宿優惠攻略|餐飲再放大1|5倍|住宿放大2倍|用在這些飯店,"['五倍券發放', '使用就在下週', '很久沒出國', '只好轉戰好的飯店放鬆CHILL一...",2021-09-30,https://ohmygodohoh.pixnet.net/blog/post/33427...,['https://pic.pimg.tw/ohmygodohoh/1632990704-1...,30005360.0,NaN
3,40000004,台北住宿推薦|台北住宿推薦|超夢幻夜景飯店|TOP10,"['台北住宿推薦', '超夢幻市景', '台北是台灣的首都', '是旅遊首選之一', '台北...",2021-09-26,https://mazda770.pixnet.net/blog/post/33421641...,['https://pic.pimg.tw/mazda770/1570324867-2843...,30005250.0,NaN
4,40000005,台北住宿|首都唯客樂飯店|松江南京老牌平價商旅|日式套房|早餐分享,"['之前住過首都大飯店旗艦館', '很喜歡這家的住宿', '很可惜去年疫情關係停業了', '...",2021-08-20,https://tloveq.pixnet.net/blog/post/47813642,['https://pic.pimg.tw/tloveq/1627810982-275640...,30000062.0,NaN


In [12]:
a.to_csv('allDatas.csv', encoding='utf-8', index=None)

In [17]:
b = df.get_pandas_df('author_feature')
b.shape

(5699, 4)

In [18]:
b.head()

,aut_id,author,tag,author_group
0,30000001,*GENISOFI*,None,None
1,30000002,000926,None,None
2,30000003,00愛哭又愛吃,None,None
3,30000004,1234,None,None
4,30000005,1313亮晶晶,None,None


In [19]:
b.to_csv('allAuthor.csv', encoding='utf-8', index=None)

In [3]:
c = df.get_pandas_df('bnb')
# c = pd.Series(c['bnb_name'])
c.to_csv('./mySQL_bnb.csv', index=None)